In [1]:
import os
import pandas as pd
import numpy as np 
from scipy.stats import zscore
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
import plotly.io as pio
import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
import inflection
from streamlit_folium import folium_static, st_folium
from haversine import haversine, Unit
from datetime import datetime
from PIL import Image
import currencyapicom

2024-11-28 12:06:51.398 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
# Reset plotly theme after import streamlit (streamlit import is causing the charts to go b&w)
pio.templates.default = 'plotly'

In [3]:
# Import dataset
dataframe = pd.read_csv('..\dataset\zomato.csv')

In [29]:
pd.set_option('display.max_columns', None)

In [95]:
client = currencyapicom.Client('cur_live_P53YtMICIpcE3WTeG82IwC6QOdZlNuGuSwAz0ioW')
result = client.latest('USD',currencies=['BWP','BRL','AED','INR','IDR','NZD','GBP','QAR','ZAR','LKR','TRY','USD'])

In [ ]:
exchange = result['data']

# Limpeza de dados

In [5]:
# Rename columns

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(' ', '')
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new

    return df


In [6]:
# Country code to country name dict 

COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}
def country_name(country_id):
    return COUNTRIES[country_id]

In [7]:
# Color code to color name dict

COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]


In [8]:
# Creating price tags

def categorize_prices(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

In [9]:
# Creating option tags

def categorize_options(column_name):
    if column_name == 1:
        return 'Yes'
    else:
        return 'No'

In [10]:
df = rename_columns(dataframe)

In [11]:
# Applying the functions defined early

df['country_code'] = df['country_code'].apply(lambda x: country_name(x))
df['rating_color'] = df['rating_color'].apply(lambda x: color_name(x))
df['price_range'] = df['price_range'].apply(lambda x: categorize_prices(x))
df['has_table_booking'] = df['has_table_booking'].apply(lambda x: categorize_options(x))
df['has_online_delivery'] = df['has_online_delivery'].apply(lambda x: categorize_options(x))
df['is_delivering_now'] = df['is_delivering_now'].apply(lambda x: categorize_options(x))



In [12]:
df.rename(columns={'country_code': 'country'}, inplace=True)

In [13]:
# Drop switch_to_order_menu column for have 0 value in all lines

df = df.drop('switch_to_order_menu', axis=1)

In [14]:
# Fill NaN values in cuisines column (affected: 15 lines)

df['cuisines'] = df['cuisines'].fillna('Not informed')

In [15]:
# Choosing only the first cuisine for multiple cuisine restaurants, CEO request
 
df['cuisines'] = df['cuisines'].apply(lambda x: x.split(",")[0])

In [16]:
# Verifying duplicated ids

df[df.duplicated(subset=['restaurant_id'], keep=False)]

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),Yes,No,No,expensive,4.6,darkgreen,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),Yes,No,No,expensive,4.6,darkgreen,Excellent,619
35,18442143,Mama Lou's Italian Kitchen,Philippines,Pasig City,"Second Floor, Ayala Malls The 30th, Meralco Av...","Ayala Malls The 30th, Ugong, Pasig City","Ayala Malls The 30th, Ugong, Pasig City, Pasig...",121.064292,14.580981,Italian,1100,Botswana Pula(P),Yes,No,No,expensive,4.5,darkgreen,Excellent,287
36,18442143,Mama Lou's Italian Kitchen,Philippines,Pasig City,"Second Floor, Ayala Malls The 30th, Meralco Av...","Ayala Malls The 30th, Ugong, Pasig City","Ayala Malls The 30th, Ugong, Pasig City, Pasig...",121.064292,14.580981,Italian,1100,Botswana Pula(P),Yes,No,No,expensive,4.5,darkgreen,Excellent,287
38,6309635,Frankie's New York Buffalo Wings,Philippines,Pasig City,"Ground Floor, City Golf Plaza, Julia Vargas Av...","City Golf Plaza, Ugong, Pasig City","City Golf Plaza, Ugong, Pasig City, Pasig City",121.067810,14.584646,American,850,Botswana Pula(P),No,No,No,expensive,4.5,darkgreen,Excelente,606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,5927402,Walter's Coffee Roastery,Turkey,İstanbul,"Cafeağa Mahallesi, Bademaltı Sokak, No 21/B, K...",Moda,"Moda, İstanbul",29.026046,40.984729,Cafe,60,Turkish Lira(TL),No,No,No,expensive,4.2,green,Very Good,967
7496,18232014,Paul's Homemade Pasta & Lasagna,Turkey,İstanbul,"Caferağa Mahallesi, Dr. Esat Işık Caddesi, No ...",Moda,"Moda, İstanbul",29.028463,40.983702,Italian,90,Turkish Lira(TL),Yes,No,No,expensive,4.4,green,Çok iyi,706
7497,18232014,Paul's Homemade Pasta & Lasagna,Turkey,İstanbul,"Caferağa Mahallesi, Dr. Esat Işık Caddesi, No ...",Moda,"Moda, İstanbul",29.028463,40.983702,Italian,90,Turkish Lira(TL),Yes,No,No,expensive,4.4,green,Çok iyi,706
7517,18384475,Happy Moon's,Turkey,İstanbul,"Watergarden Yaşam Merkezi, Barbaros Mahallesi,...","Watergarden, Batı Ataşehir, Ataşehir","Watergarden, Batı Ataşehir, Ataşehir, İstanbul",29.099755,40.998032,World Cuisine,200,Turkish Lira(TL),No,No,No,gourmet,4.4,green,Very Good,503


In [17]:
# Droping duplicated ids

df.drop_duplicates(subset='restaurant_id', keep='first', inplace=True)

In [18]:
# Droping outliner value from column 'average_cost_for_two' for Australia country

df = df.loc[df['average_cost_for_two'] != 25000017, :]

In [19]:
# Currency names to currency code dict 

CODES = {
'Botswana Pula(P)' : 'BWP',
'Brazilian Real(R$)' : 'BRL',
'Dollar($)' : 'USD',
'Emirati Diram(AED)' : 'AED',
'Indian Rupees(Rs.)' : 'INR',
'Indonesian Rupiah(IDR)' : 'IDR',
'NewZealand($)' : 'NZD',
'Pounds(£)' : 'GBP',
'Qatari Rial(QR)' : 'QAR',
'Rand(R)' : 'ZAR',
'Sri Lankan Rupee(LKR)' : 'LKR',
'Turkish Lira(TL)' : 'TRY'
}       

def currency_code(currency):
    return CODES[currency]

In [20]:
VALUES = {    
'BWP' : 0.07346,
'BRL' : 0.17196,
'USD' : 1.0,
'AED' : 0.27229,
'INR' : 0.01182,
'IDR' : 0.000062795,
'NZD' : 0.58599,
'GBP' : 1.25916,
'QAR' : 0.27472,
'ZAR' : 0.05520,
'LKR' : 0.00343,
'TRY' : 0.02894   
}

def usd_value(currency_value):
    return VALUES[currency_value]

In [21]:
#Creating the currency code tag by applying a lambda function within the dict CODES

df['currency'] = df['currency'].apply(lambda x: currency_code(x))
df.rename(columns= {'currency' : 'currency_codes'}, inplace=True)

In [22]:
df['currency_values'] = df['currency_codes'].apply(lambda x: usd_value(x))

In [23]:
# Defining the lambda function to convert local currency to usd

to_usd = lambda x: (x['average_cost_for_two'] * x['currency_values'])
df['currency_in_usd'] = df.apply(to_usd, axis = 1)

In [24]:
# Changing the column order for clarity

df = df.reindex(['restaurant_id', 'restaurant_name', 'country', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency_codes', 'currency_in_usd', 
       'has_table_booking', 'has_online_delivery', 'is_delivering_now', 'price_range',
       'aggregate_rating', 'rating_color', 'rating_text', 'votes'], axis=1)

In [25]:
df.head(2)

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency_codes,currency_in_usd,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,BWP,80.806,Yes,No,No,expensive,4.6,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,3100,BWP,227.726,No,No,No,gourmet,4.7,darkgreen,Excellent,469


# 1.0. General insights:

## 1. Quantos restaurantes únicos estão registrados?


In [42]:
unique_restaurants = df['restaurant_id'].count()
unique_restaurants

6941

## 2. Quantos países únicos estão registrados? 

In [43]:
unique_countries = df['country'].nunique()
unique_countries

15

## 3. Quantas cidades únicas estão registradas?


In [44]:
city_unique = df['city'].nunique()
city_unique

125

## 4. Qual o total de avaliações feitas?


In [45]:
total_votes = df['votes'].sum()
total_votes

4195431

## 5. Qual o total de tipos de culinária registrados?

In [46]:
cuisines_count = df['cuisines'].nunique()
cuisines_count

166

# 2.0. Country insights:

## 1. Qual o nome do país que possui mais cidades registradas?


In [49]:
city_by_country = df.loc[:, ['country','city']].groupby(['country']).nunique().sort_values('city', ascending=False).reset_index()
city_by_country

,country,city
0,India,49
1,United States of America,22
2,Philippines,12
3,South Africa,12
4,England,5
5,New Zeland,4
6,United Arab Emirates,4
7,Australia,3
8,Brazil,3
9,Canada,3


## 2. Qual o nome do país que possui mais restaurantes registrados?


In [50]:
restaurants_by_country = df.loc[:, ['country','restaurant_id']].groupby(['country']).count().sort_values('restaurant_id', ascending=False).reset_index()
restaurants_by_country

,country,restaurant_id
0,India,3120
1,United States of America,1378
2,England,400
3,South Africa,346
4,United Arab Emirates,300
5,Brazil,240
6,New Zeland,239
7,Canada,180
8,Australia,179
9,Turkey,159


## 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?



In [52]:
price_range_4 = df.loc[df['price_range'] == 'gourmet', ['country', 'restaurant_id']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index()
price_range_4

,country,restaurant_id
0,United States of America,417
1,South Africa,221
2,India,195
3,Brazil,148
4,United Arab Emirates,102
5,England,79
6,New Zeland,79
7,Turkey,59
8,Singapure,58
9,Canada,54


## 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?


In [53]:
unique_cuisines_country = df.loc[:, ['country','cuisines']].groupby('country').nunique().sort_values('cuisines', ascending=False).reset_index()
unique_cuisines_country

,country,cuisines
0,India,78
1,United States of America,75
2,England,52
3,United Arab Emirates,46
4,Australia,43
5,Brazil,43
6,Canada,42
7,South Africa,42
8,New Zeland,37
9,Qatar,31


## 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?


In [54]:
votes_by_country = df.loc[:, ['country','votes']].groupby('country').sum().sort_values('votes', ascending=False).reset_index()
votes_by_country

,country,votes
0,India,2800199
1,United States of America,523414
2,United Arab Emirates,177564
3,Australia,130154
4,Canada,105018
5,Turkey,100193
6,Indonesia,89026
7,South Africa,81939
8,New Zeland,52532
9,Philippines,48398


## 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?


In [91]:
most_restaurant_delivery = df.loc[df['is_delivering_now'] == 'Yes', ['restaurant_id','country']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index()
most_restaurant_delivery

,country,restaurant_id
0,India,1150
1,United Arab Emirates,52
2,Philippines,7
3,Qatar,3


## 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?


In [57]:
most_booking_restaurants = df.loc[df['has_table_booking'] == 'Yes', ['restaurant_id','country']].groupby('country').count().sort_values('restaurant_id', ascending=False).reset_index()
most_booking_restaurants

,country,restaurant_id
0,India,256
1,England,55
2,Australia,28
3,Philippines,22
4,New Zeland,19
5,Indonesia,14
6,United Arab Emirates,10
7,Qatar,4
8,Turkey,4
9,South Africa,3


## 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?


In [58]:
avg_votes = df.loc[:,['votes', 'country']].groupby('country').mean().sort_values('votes', ascending=False).reset_index()
np.round(avg_votes, 2)

,country,votes
0,Indonesia,1112.82
1,India,897.50
2,Australia,727.12
3,Turkey,630.14
4,Philippines,604.98
5,United Arab Emirates,591.88
6,Canada,583.43
7,United States of America,379.84
8,Qatar,376.32
9,South Africa,236.82


## 9. Qual o nome do país que possui, na média, a maior nota média registrada?


In [59]:
avg_rating = df.loc[:,['aggregate_rating', 'country']].groupby('country').mean().sort_values('aggregate_rating', ascending=False).reset_index()
np.round(avg_rating, 2)

,country,aggregate_rating
0,Indonesia,4.60
1,Philippines,4.46
2,Singapure,4.44
3,United States of America,4.40
4,Australia,4.37
5,Canada,4.32
6,Turkey,4.31
7,Qatar,4.24
8,New Zeland,4.16
9,South Africa,4.06


## 10. Qual o nome do país que possui, na média, a menor nota média registrada?


In [60]:
avg_rating = df.loc[:,['aggregate_rating', 'country']].groupby('country').mean().sort_values('aggregate_rating', ascending=True).reset_index()
np.round(avg_rating, 2)

,country,aggregate_rating
0,Brazil,3.32
1,United Arab Emirates,4.02
2,India,4.03
3,England,4.04
4,Sri Lanka,4.06
5,South Africa,4.06
6,New Zeland,4.16
7,Qatar,4.24
8,Turkey,4.31
9,Canada,4.32


## 11. Qual a média de preço de um prato para dois por país?

In [92]:
df.head(1)

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency_codes,currency_in_usd,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,BWP,81.082661,Yes,No,No,expensive,4.6,darkgreen,Excellent,619


In [93]:
avg_cost_for_two = df.loc[:, ['currency_in_usd','country', 'currency_codes']].groupby(['country','currency_codes']).mean().reset_index()
avg_cost_for_two

,country,currency_codes,currency_in_usd
0,Australia,USD,71.195531
1,Brazil,BRL,24.078905
2,Canada,USD,41.861111
3,England,GBP,55.419684
4,India,INR,8.338071
5,Indonesia,IDR,19.206775
6,New Zeland,NZD,36.813496
7,Philippines,BWP,90.504834
8,Qatar,QAR,47.752600
9,Singapure,USD,141.437500


# 3.0. City insights

In [44]:
price_range_city = df.loc[df['price_range'] == 'cheap', ['price_range','city']].groupby('city').count().sort_values('price_range', ascending= False).reset_index()
price_range_city
# price_range_top = price_range_city.loc[0,'price_range']
# price_range_top

,city,price_range
0,Ludhiana,38
1,Bhubaneshwar,35
2,Gangtok,32
3,Ooty,32
4,Ranchi,32
...,...,...
80,Portland,1
81,Sandton,1
82,Ahmedabad,1
83,Brisbane,1


## 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [45]:
city_rest_count = df.loc[:,['restaurant_id','city']].groupby('city').count().sort_values('restaurant_id', ascending=False)
city_rest_count.columns = ['rest_count']
city_rest_count.reset_index()

,city,rest_count
0,Abu Dhabi,80
1,Kanpur,80
2,Mangalore,80
3,Manchester,80
4,Lucknow,80
...,...,...
120,Marikina City,1
121,Ghaziabad,1
122,Gandhinagar,1
123,Muntinlupa City,1


In [46]:
filtered_cities = city_rest_count.loc[city_rest_count['rest_count'] == 80,:].reset_index()
filtered_cities.head(3)

,city,rest_count
0,Abu Dhabi,80
1,Kanpur,80
2,Mangalore,80


In [47]:
min_ids = df.loc[df['city'].isin(filtered_cities['city']), ['restaurant_id', 'city']].groupby('city').min().reset_index()
min_ids.sort_values('restaurant_id', ascending=True)

min_ids
# top_reg_rest = min_ids.loc[44,['city']]
# top_reg_rest

,city,restaurant_id
0,Abu Dhabi,5700005
1,Agra,3400005
2,Allahabad,2400005
3,Amritsar,2200000
4,Ankara,6000019
5,Aurangabad,2500005
6,Austin,16911952
7,Bangalore,50066
8,Bhopal,2600003
9,Bhubaneshwar,2900004


## 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [103]:
city_rating_above = df.loc[df['aggregate_rating'] > 4.0 , ['city', 'restaurant_id']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
# city_best_rating = city_rating_above.loc[0, ['city']]
# city_best_rating
city_rating_above

,city,restaurant_id
0,Bangalore,79
1,London,78
2,Houston,75
3,Jakarta,74
4,Chennai,72
...,...,...
116,Ghaziabad,1
117,San Juan City,1
118,Roodepoort,1
119,Zirakpur,1


## 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?


In [157]:
city_rating_below = df.loc[df['aggregate_rating'] < 2.5 , ['city', 'restaurant_id']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
city_worst_rating = city_rating_below.loc[0, ['city']]
city_worst_rating


city    Gangtok
Name: 0, dtype: object

## 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?


In [160]:
cost_city = np.round(df.loc[:, ['average_cost_for_two', 'city']].groupby('city').mean().sort_values('average_cost_for_two', ascending=False).reset_index(), 3)
expensive_cost_city = cost_city.loc[0, ['city']]
expensive_cost_city

city    Jakarta
Name: 0, dtype: object

## 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?


In [169]:
cuisines_city = df.loc[:, ['cuisines', 'city']].groupby('city').nunique().sort_values('cuisines', ascending=False).reset_index()
most_cuisines_city = cuisines_city.loc[0, ['city']]
most_cuisines_city

city    Birmingham
Name: 0, dtype: object

## 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?


In [ ]:
booking_city = df.loc[df['has_table_booking'] == 'Yes', ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
most_booking_city = booking_city.loc[0, ['city']]
most_booking_city

city    Bangalore
Name: 0, dtype: object

## 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?


In [ ]:
delivery_city = df.loc[df['is_delivering_now'] == 'Yes', ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
delivery_city.columns = ['city', 'rest_count']
delivery_city.head(5)


In [ ]:
filter = delivery_city.loc[delivery_city['rest_count'] == 48,:]
filter

In [187]:
older_id_result = df.loc[ df['city'].isin(filter['city']) ,  ['restaurant_id', 'city']].groupby('city').min().sort_values('restaurant_id', ascending=True).reset_index()
most_delivery_city = older_id_result.loc[0, ['city']]
most_delivery_city

city    Amritsar
Name: 0, dtype: object

## 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [189]:
online_delivery_city = df.loc[df['has_online_delivery'] == 'Yes', ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending=False).reset_index()
most_online_delivery_city = online_delivery_city.loc[0, ['city']]
most_online_delivery_city

city    Bhopal
Name: 0, dtype: object

# 4.0. Restaurant insights

## 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [195]:
df.loc[:, ['restaurant_name','votes']].sort_values(['votes'], ascending=False)

,restaurant_name,votes
3928,Bawarchi,41333
3038,Byg Brewski Brewing Company,17394
2986,Toit,15270
3002,Truffles,14984
5076,Hauz Khas Social,13627
...,...,...
333,Barbosa,0
285,Cantinho da Gula,0
165,Capricho Mineiro,0
5165,Kabab Corner,0


## 2. Qual o nome do restaurante com a maior nota média?

In [196]:
df.loc[:, ['restaurant_name','aggregate_rating']].sort_values(['aggregate_rating'], ascending=False)

,restaurant_name,aggregate_rating
792,Kuma's Corner - Belmont,4.9
4940,Barbeque Nation,4.9
1697,The Parlor Pizzeria,4.9
825,Xoco,4.9
822,Portillo's Hot Dogs,4.9
...,...,...
6751,Alston Bar & Beef,0.0
151,Galeteria Beira Lago,0.0
6749,Ziya Asian Grill,0.0
6748,Veeno,0.0


## 3. Qual o nome do restaurante que possui o maior valor de um prato para duas pessoas?

In [95]:
df.loc[:, ['restaurant_name','currency_in_usd']].sort_values(['currency_in_usd'], ascending=False)

,restaurant_name,currency_in_usd
2121,Corner House,755.0
1468,Eleven Madison Park,650.0
798,Alinea,600.0
1450,Per Se,600.0
2050,Raffles Grill,500.0
...,...,...
1112,Shiva Indian,0.0
1135,Tiny Boxwood's,0.0
1240,Eggslut,0.0
1517,The Albion Rooms,0.0


## 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [211]:
df.loc[df['cuisines'] == 'Brazilian', ['restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating'],ascending=True).head(10)

,restaurant_name,aggregate_rating
166,NAVE MÃE FAST FOOD GOURMET,0.0
335,Tempero das Gerais,0.0
318,Bar do Luiz Fernandes,0.0
285,Cantinho da Gula,0.0
220,Quitéria,0.0
186,Rancho Inn,0.0
167,Restaurante dos Amigos,0.0
333,Barbosa,0.0
165,Capricho Mineiro,0.0
319,Famiglia Rivitti,0.0


## 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [210]:
df.loc[(df['cuisines'] == 'Brazilian') & (df['country'] == 'Brazil'), ['restaurant_name', 'aggregate_rating']].sort_values(['aggregate_rating'],ascending=False).head(10)

,restaurant_name,aggregate_rating
207,Braseiro da Gávea,4.9
254,Aprazível,4.9
346,Mocotó,4.8
215,Garota de Ipanema,4.8
179,Fogo de Chão,4.8
261,Aconchego Carioca,4.7
217,Zazá Bistrô Tropical,4.7
347,Veloso,4.6
213,Balada Mix,4.6
235,Giuseppe Grill Mar,4.5


## 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [263]:
yes_delivery = df.loc[df['has_online_delivery'] == 'Yes', ['restaurant_name', 'votes']].sort_values(['votes'], ascending=False)
yes_delivery['votes'].mean()


838.8216639209226

In [264]:
no_delivery = df.loc[df['has_online_delivery'] == 'No', ['restaurant_name', 'votes']].sort_values(['votes'], ascending=False)
no_delivery['votes'].mean()

478.3452249058276

## 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [96]:
df.head(1)

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency_codes,currency_in_usd,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,BWP,81.082661,Yes,No,No,expensive,4.6,darkgreen,Excellent,619


In [98]:
yes_booking = df.loc[df['has_table_booking'] == 'Yes', ['restaurant_name','currency_in_usd']].sort_values(['currency_in_usd'], ascending=False)
yes_booking['currency_in_usd'].mean()

35.66111836184717

In [99]:
yes_booking = df.loc[df['has_table_booking'] == 'No', ['restaurant_name','currency_in_usd']].sort_values(['currency_in_usd'], ascending=False)
yes_booking['currency_in_usd'].mean()

28.849644167680076

## 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [100]:
jp_usa = df.loc[(df['cuisines'] == 'Japanese') & (df['country'] == 'United States of America'), ['restaurant_name','currency_in_usd']].sort_values(['currency_in_usd'], ascending=False)
jp_usa['currency_in_usd'].mean()

56.40625

In [101]:
bbq_usa = df.loc[(df['cuisines'] == 'BBQ') & (df['country'] == 'United States of America'), ['restaurant_name','currency_in_usd']].sort_values(['currency_in_usd'], ascending=False)
bbq_usa['currency_in_usd'].mean()


39.642857142857146

# 5.0. Cuisines insight

## 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [21]:
df.loc[df['cuisines'] == 'Italian',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False)

,restaurant_name,aggregate_rating
366,Andre's Cucina & Polenta Bar,4.9
6305,Ombra,4.9
1280,Perricone's Marketplace & Café,4.9
5408,Darshan,4.9
4056,Zolocrust - Hotel Clarks Amer,4.9
...,...,...
6723,Le Delicatezze Di Bruno,0.0
6748,Veeno,0.0
242,Bene - Sheraton Rio Hotel,0.0
6752,The Pasta Factory,0.0


## 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?


In [22]:
df.loc[df['cuisines'] == 'Italian',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=True)

,restaurant_name,aggregate_rating
6723,Le Delicatezze Di Bruno,0.0
324,Più,0.0
305,Ristorantino,0.0
242,Bene - Sheraton Rio Hotel,0.0
184,La Bocca Bar e Trattoria,0.0
...,...,...
1805,Cerroni's Purple Garlic,4.9
800,Chicago Pizza & Oven Grinder Company,4.9
6574,Celino's,4.9
7233,Cafe Del Sol Classico,4.9


## 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?


In [23]:
df.loc[df['cuisines'] == 'American',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False)

,restaurant_name,aggregate_rating
726,OEB Breakfast Co.,4.9
1135,Tiny Boxwood's,4.9
939,"Tocabe, An American Indian Eatery",4.9
422,Brick Store Pub,4.9
1019,Union Woodshop,4.9
...,...,...
6348,Thunderbird Cafe,3.2
7261,Hard Rock Cafe,2.8
251,Bob's,2.6
7204,Guy Fieri's Kitchen & Bar,2.2


## 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?


In [24]:
df.loc[df['cuisines'] == 'American',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=True)

,restaurant_name,aggregate_rating
6751,Alston Bar & Beef,0.0
7204,Guy Fieri's Kitchen & Bar,2.2
251,Bob's,2.6
7261,Hard Rock Cafe,2.8
6348,Thunderbird Cafe,3.2
...,...,...
822,Portillo's Hot Dogs,4.9
726,OEB Breakfast Co.,4.9
599,Fat Cat,4.9
1918,Kono's Surf Club Cafe,4.9


## 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [29]:
df.loc[df['cuisines'] == 'Arabian',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False).head(5)

,restaurant_name,aggregate_rating
3896,Mandi@36,4.7
6883,Wok of Fame,4.5
3184,Three Kings,4.5
2452,Madfoon Al Khaimah,4.5
2303,Aroos Damascus,4.4


## 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?


In [30]:
df.loc[df['cuisines'] == 'Arabian',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=True).head(5)

,restaurant_name,aggregate_rating
337,Raful,0.0
169,Empório Árabe,2.7
147,Salim Sou Eu,3.1
5118,Abad Hot Chicken,3.2
5155,V Empire,3.2


## 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?


In [31]:
df.loc[df['cuisines'] == 'Japanese',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False).head(5)

,restaurant_name,aggregate_rating
240,Sushi Leblon,4.9
75,Mendokoro Ramenba,4.9
1297,Samurai,4.9
1567,Nobu Perth,4.9
6034,Gyu - Kaku Japanese BBQ,4.9


## 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?


In [32]:
df.loc[df['cuisines'] == 'Japanese',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=True).head(5)

,restaurant_name,aggregate_rating
123,Banzai Sushi,0.0
181,Orienthai,2.4
182,Hachiko,2.5
112,Haná,2.7
116,Sushi Loko,2.9


## 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?


In [35]:
df.loc[df['cuisines'] == 'Home-made',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False)


,restaurant_name,aggregate_rating
7525,Kanaat Lokantası,4.0
7373,GurMekan Restaurant,3.7


## 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?


In [36]:
df.loc[df['cuisines'] == 'Home-made',['restaurant_name', 'aggregate_rating']].sort_values('aggregate_rating', ascending=True)


,restaurant_name,aggregate_rating
7373,GurMekan Restaurant,3.7
7525,Kanaat Lokantası,4.0


## 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?


In [102]:
df.loc[:, ['cuisines','currency_in_usd']].sort_values('currency_in_usd', ascending=False).head(5)

,cuisines,currency_in_usd
2121,Continental,755.0
1468,New American,650.0
798,New American,600.0
1450,French,600.0
2050,French,500.0


## 12. Qual o tipo de culinária que possui a maior nota média?


In [40]:
df.loc[:, ['cuisines','aggregate_rating']].sort_values('aggregate_rating', ascending=False).head(5)

,cuisines,aggregate_rating
792,Burger,4.9
4940,North Indian,4.9
1697,Italian,4.9
825,Mexican,4.9
822,American,4.9


## 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [43]:
df.loc[(df['is_delivering_now'] == 'Yes') & (df['has_online_delivery'] == 'Yes') , ['restaurant_id','cuisines']].groupby('cuisines').count().sort_values('restaurant_id', ascending=False).reset_index().head(10)


,cuisines,restaurant_id
0,North Indian,317
1,Cafe,131
2,South Indian,79
3,Burger,76
4,Fast Food,72
5,Pizza,55
6,Chinese,49
7,Biryani,38
8,Italian,35
9,Continental,34


# Gráficos

## 1. Mapa

In [ ]:
df_aux = df.loc[:,['longitude','latitude', 'city', 'restaurant_name', 'rating_color']]

m = folium.Map()

mark_cluster = MarkerCluster().add_to(m)

for index, row in df_aux.iterrows():
    folium.Marker(location = [row['latitude'], 
                              row['longitude']],
                              popup = row[['city','restaurant_name']],
                              icon=folium.Icon(color=row['rating_color']),
                              ).add_to(mark_cluster)
    
folium.LayerControl().add_to(m)

m

In [ ]:
df_aux = df.loc[:,['longitude','latitude', 'city', 'restaurant_name', 'rating_color', 'country']]

m = folium.Map()

mark_cluster = MarkerCluster().add_to(m)

df_aux.apply(lambda x: folium.Marker(location = [x['latitude'], 
                              x['longitude']],
                              popup = x[['city','restaurant_name']],
                              icon=folium.Icon(color=x['rating_color']),
                              ).add_to(mark_cluster), axis= 1)
   
folium.LayerControl().add_to(m)

m


In [94]:
df_aux = df.loc[:,['longitude','latitude', 'city', 'restaurant_name', 'rating_color']]

callback = """\
    function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
    icon: "map-marker", markerColor: 'red'});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

m = folium.Map()

FastMarkerCluster(data=list(zip(df_aux['latitude'], df_aux['longitude'])), callback = callback).add_to(m)

m


## 2. Número de restaurantes por país

In [28]:
df_aux01 = df.loc[:, ['restaurant_id','country']].groupby(['country']).count().sort_values('restaurant_id', ascending=False).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(name='Restaurants_by_country', 
                             x=df_aux01['country'], 
                             y=df_aux01['restaurant_id']),
                             )
fig.update_layout(barmode= 'group')
fig.update_layout(autosize= False, width= 1200, height= 600)
fig.update_yaxes(automargin = True)
fig.update_layout(xaxis_title = 'Countries', yaxis_title = 'Restaurant count')
fig.update_traces(textposition='outside', selector=dict(type='bar'))
fig.update_traces(text= df_aux01['restaurant_id'], selector=dict(type='bar'))
fig.show()

## 3. Número de cidades por país 

In [48]:
grouped = df.loc[:, ['city','country']].groupby('country').nunique().sort_values('city', ascending=False).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(name='city_by_country',
                     x= grouped['country'],
                     y= grouped['city']))

fig.update_layout(autosize= False, width=1200, height= 600)
fig.update_yaxes(automargin = True)
fig.update_layout(xaxis_title= 'Countries', yaxis_title= 'City count')
fig.update_traces(textposition= 'outside')
fig.update_traces(text= grouped['city'])


fig.show()


## 4. Top 10 países com maiores notas

In [27]:
avg_rating = np.round(df.loc[:,['aggregate_rating', 'country']].groupby('country').mean().sort_values('aggregate_rating', ascending=True).reset_index(), 2).head(10)
avg_votes = df.loc[:,['votes', 'country']].groupby('country').mean().sort_values('votes', ascending=False).reset_index()

colors1 = ['#636efa'] * 7 + ['#c7c704', '#878901', '#4d4f00'] 

fig = go.Figure()

fig.add_trace(go.Bar(name='top_best_rating', 
                     x= avg_rating['aggregate_rating'], 
                     y= avg_rating['country'],
                     orientation= 'h',
                     marker_color = colors1))

fig.update_layout(autosize = False, width= 1200, height= 600)
fig.update_yaxes(automargin= True)
fig.update_layout(xaxis_title= 'Countries', yaxis_title= 'Avg. Ratings')
fig.update_traces(text= avg_rating['aggregate_rating'])
fig.update_traces(textposition = 'outside')

fig.show()


## 5. Top 10 países com mais avaliações

In [28]:
avg_votes = np.round(df.loc[:,['votes', 'country']].groupby('country').mean().sort_values('votes', ascending=True).reset_index(), 2).head(10)

colors2 = ['#636efa'] * 7 + ['#c7c704', '#878901', '#4d4f00'] 

fig = go.Figure()

fig.add_trace(go.Bar(name='top_votes',
                     x= avg_votes['votes'],
                     y= avg_votes['country'],
                     orientation= 'h',
                     marker_color = colors2))

fig.update_layout(autosize= False, width= 1200, height= 600)
fig.update_yaxes(automargin = True)
fig.update_layout(xaxis_title = 'Avg. Votes', yaxis_title= 'Countries')
fig.update_traces(text= avg_votes['votes'])
fig.update_traces(textposition = 'outside')

fig.show()




## 6. média de preço do prato para dois em dólar por país

In [29]:
grouped = np.round(df.loc[:, ['currency_in_usd','country']].groupby('country').mean('currency_in_usd').sort_values('currency_in_usd', ascending=False).reset_index(), 2)

fig = go.Figure()

fig.add_trace(go.Bar(name='avg_price_by_country', 
                     x= grouped['country'], 
                     y= grouped['currency_in_usd']))

fig.update_layout(autosize = False, width= 1200, height= 600)
fig.update_yaxes(automargin = True)
fig.update_layout(xaxis_title= 'Countries', yaxis_title= 'Avg. cost for two (USD)')
fig.update_traces(text= grouped['currency_in_usd'])
fig.update_traces(textposition= 'outside')




## 7. Top 10 cidades com mais restaurantes registrados

In [ ]:
grouped = df.loc[:, ['restaurant_id','city', 'country']].groupby(['country','city']).count().sort_values('restaurant_id', ascending=False).reset_index().head(10)

fig = px.bar(grouped, x= 'city', y= 'restaurant_id', color= 'country', text_auto= True)
fig.update_layout(xaxis_title= 'Cities', yaxis_title= 'Restaurant count')
fig.update_layout(xaxis={'categoryorder':'total descending'})


## 8. Top 10 cidades com nota > 4

In [143]:
city_rating_above = df.loc[df['aggregate_rating'] > 4.0 , ['city', 'restaurant_id', 'country']].groupby(['city', 'country']).count().sort_values('restaurant_id', ascending=False).reset_index().head(10)

fig = px.bar(city_rating_above, x= 'city', y= 'restaurant_id', color= 'country', text_auto=True)
fig.update_traces(textposition= 'inside')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(xaxis_title= 'Cities', yaxis_title= 'Restaurant count')


fig.show()

## 9. Top 10 cidades com nota > 2.5

In [59]:
city_rating_below = df.loc[df['aggregate_rating'] < 2.5 , ['city', 'restaurant_id', 'country']].groupby(['city', 'country']).count().sort_values('restaurant_id', ascending=False).reset_index().head(10)

fig = px.bar(city_rating_below, x= 'city', y= 'restaurant_id', color= 'country', text_auto=True)
fig.update_traces(textposition= 'outside')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(xaxis_title= 'Cities', yaxis_title= 'Restaurant count')


fig.show()


## 10. Tipos distintos de culinária por país

In [60]:
df.head(2)

,restaurant_id,restaurant_name,country,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency_codes,currency_in_usd,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,BWP,80.910004,Yes,No,No,expensive,4.6,darkgreen,Excellent,619
2,6314542,Blackbird,Philippines,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,3100,BWP,228.019101,No,No,No,gourmet,4.7,darkgreen,Excellent,469


In [67]:
grouped = df.loc[:,['cuisines','country']].groupby('country').nunique().sort_values('cuisines', ascending=True).reset_index()

colors2 = ['#636efa'] * 12 + ['#c7c704', '#878901', '#4d4f00'] 

fig = go.Figure()

fig.add_trace(go.Bar(name='cuisines_by_country',
                     x= grouped['cuisines'],
                     y= grouped['country'],
                     orientation= 'h',
                     marker_color = colors2))

fig.update_layout(autosize= False, width= 1200, height= 600)
fig.update_yaxes(automargin = True)
fig.update_layout(xaxis_title = 'Distinct cuisines count', yaxis_title= 'Countries')
fig.update_traces(text= grouped['cuisines'])
fig.update_traces(textposition = 'outside')

fig.show()

## 11. Top 10 cidades com mais tipos diferentes de culinária

In [69]:
grouped = df.loc[:,['cuisines','city', 'country']].groupby(['city','country']).nunique().sort_values('cuisines', ascending=False).reset_index().head(10)

fig = px.bar(grouped, x= 'city', y= 'cuisines', color= 'country', text_auto=True)
fig.update_traces(textposition= 'outside')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(xaxis_title= 'Cities', yaxis_title= 'Restaurant count')


## 12. Top 10 melhores restaurantes

In [92]:
grouped = df.loc[:,['restaurant_name', 'aggregate_rating', 'votes']].groupby(['restaurant_name']).mean().sort_values(['aggregate_rating','votes'], ascending=False).reset_index()
grouped.columns = ['Restaurant name','Avg. rating','Avg. votes']
grouped

,Restaurant name,Avg. rating,Avg. votes
0,Spice Kraft,4.9,4935.0
1,Café Du Monde,4.9,4036.0
2,Coal Barbecues,4.9,3646.5
3,Indian Grill Room,4.9,3062.0
4,Tapri Central,4.9,2862.0
...,...,...,...
5908,Le Bistrot du Cuisinier,0.0,0.0
5909,Tempero das Gerais,0.0,0.0
5910,The Pasta Factory,0.0,0.0
5911,Veeno,0.0,0.0


## 13. Top 10 culinárias populares

In [106]:
grouped = df.loc[:, ['cuisines', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False)
grouped.columns = ['Cuisines','Avg. rating']
grouped



,Cuisines,Avg. rating
792,Burger,4.9
4940,North Indian,4.9
1697,Italian,4.9
825,Mexican,4.9
822,American,4.9
...,...,...
6751,American,0.0
151,Brazilian,0.0
6749,Indian,0.0
6748,Italian,0.0


## 14. Gráfico de pizza, porcentagem de delivery, online delivery e booking

* booking

In [29]:
df_aux = df.loc[:, ['has_table_booking', 'restaurant_id']].groupby('has_table_booking').count().reset_index()

df_aux['booking_percent'] = (df_aux['restaurant_id'] / df_aux['restaurant_id'].sum()) * 100

fig = go.Figure(data=[go.Pie(labels= df_aux['has_table_booking'], values= df_aux['booking_percent'], pull= [0, 0.1, 0])])
fig




* online delivery

In [41]:
df_aux = df.loc[:, ['has_online_delivery', 'restaurant_id']].groupby('has_online_delivery').count().reset_index()

df_aux['booking_percent'] = (df_aux['restaurant_id'] / df_aux['restaurant_id'].sum()) * 100

fig = go.Figure(data=[go.Pie(labels= df_aux['has_online_delivery'], values= df_aux['booking_percent'], pull= [0, 0.06, 0])])
fig

* delivery

In [36]:
df_aux = df.loc[:, ['is_delivering_now', 'restaurant_id']].groupby('is_delivering_now').count().reset_index()

df_aux['booking_percent'] = (df_aux['restaurant_id'] / df_aux['restaurant_id'].sum()) * 100

fig = go.Figure(data=[go.Pie(labels= df_aux['is_delivering_now'], values= df_aux['booking_percent'], pull= [0, 0.1, 0])])
fig